In [ ]:
import backtrader as bt
import pandas as pd
import yfinance as yf
import numpy as np

from utils.tester import log_results, test_strategy
from utils.strategies import EMASwing

## Source data locally

In [ ]:
data_local = pd.read_csv("../data/xauusd_1h.csv", index_col=None)
data_local['datetime'] = pd.to_datetime(data_local['datetime'])
print(data_local.head())

data_feed_local = bt.feeds.PandasData(dataname=data_local, name="xau",
                                      datetime=0, openinterest=-1,
                                      open=1,
                                      high=2,
                                      low=3,
                                      close=4,
                                      volume=5)

## Download Data from YF

In [ ]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2023-06-01', 'end': '2025-01-01',
             'interval': '1h', 'multi_level_index': False, 'auto_adjust': True}
data_names = ['^RUT', '^IXIC', '^DJI', '^GSPC', '^VIX', 'GOOG', 'MSFT', 'AMZN', 'TSLA', 
              'NFLX', 'NVDA', 'AMD']

datas = dict()
for name in data_names:
    datas[name] = yf.download(tickers=name, **yf_params)


for name, data in datas.items():
    data.reset_index(inplace=True)
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    print(f"{name}: from {data['Datetime'].iloc[0]} to {data['Datetime'].iloc[-1]}")

data_feeds = dict()
for name, data in datas.items():
    data_feeds[name] = bt.feeds.pandafeed.PandasData(
        name=name,
        dataname=data,
        datetime=0,
        openinterest=-1,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
    )

# Start Backtesting

In [ ]:
strategy_params = dict(ema_period=21,
                       atr_period=9,
                       atr_multiplier=1.5,
                       rrr=1.5,
)
results = dict()

for name, data_feed in data_feeds.items():
    results[name], cereb = test_strategy(EMASwing, data_feed, strategy_params)

In [ ]:
for name, res in results.items():
    log_results(name, res)